# SQLAlchemy Joins

## Setup

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [10]:
engine = create_engine("sqlite:///mammal_masses.sqlite", echo=False)

In [11]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['ea', 'na']

In [12]:
# Map Europe class
EA = Base.classes.ea

In [13]:
# Map North American class
NA = Base.classes.na

In [14]:
# create a session
session = Session(engine)

## Filtering Review

Filters are the "WHERE" clause for your select statement. 

In [15]:
# filter North American mammals whose genus is "Antilocapra"
# query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print("Family: {0}, Genus: {1}".format(mammal.family, mammal.genus))

Family: Antilocapridae, Genus: Antilocapra


## Joins

A SQL join combines columns from one or more tables in a relational database. 

It creates a set that can be saved as a table or used as it is. 

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.

In [16]:
inspector = inspect(engine)
inspector.get_table_names()

['ea', 'na']

In [17]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
record_id INTEGER
continent TEXT
status TEXT
sporder TEXT
family TEXT
genus TEXT
species TEXT
log_mass_g FLOAT
comb_mass_g FLOAT
reference TEXT


In [22]:
inspector.get_foreign_keys('na')


[]

In [23]:
# Cross Join
str(session.query(EA.sporder, NA.sporder).limit(100))

'SELECT ea.sporder AS ea_sporder, na.sporder AS na_sporder \nFROM ea, na\n LIMIT ? OFFSET ?'

In [19]:
same_sporder = session.query(EA, NA).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea, na) = record
    print(ea)
    print(na)

In [11]:
# Return all animals from EA and NA belonging to the same sporder.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same sporder as the North American aniimal '{na_fam} {na_gen} {na_spec}'.")

The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Antilocapra americana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Capromeryx mexicana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Capromeryx minor'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Stockoceros conklingi'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Stockoceros onusrosagris'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Antilocapridae Tetrameryx shuleri'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American aniimal 'Bovidae Bison bis